## Installations

We first must install all required packages necessary for agent to run. Packages here are managed by `poetry`. For ease of use, we convert the poetry lock to a `requirements.txt` file. We then `pip install` the packages within this file.

Any errors are encountered during installation of any particular package can usually be resolved by manually pip installng that package individually.

In [ ]:
!poetry export --without-hashes --all-extras > requirements.txt

Some tasks require the installation of additional data or packages, run the cell below to install these.

In [ ]:
!sudo apt install build-essential cmake

## Running a custom task

Now let's go through how to integrate your own task and llm in the agent,
in this case, we use deepseek-coder as the LLM and implement and a custom task in robotics.


### Step 1: Specify task config
We must first specify a config file for the task in the /confis/task/ directory. 

For example: /configs/task/example_task.yaml

In [ ]:
example_task_yaml = '''
# @package _global_
agent:
  main_flow: ???
  prompt_builder:
    template_paths:
      - example_task
      - default

task:
  _target_: agent.tasks.example_task.Example_task

  name: example_task_test_env
  description:
  subtask: null
  version: v0.1
'''

### Step 2: Implement a custom task class 
Within /src/agent/tasks/ we define a new file titled example_task.py. We define a new Task class named Example_Task. In this new Task class instance, we can define new functions specific to our environment.

In [ ]:
import math
import re
import time
from datasets import load_dataset
import subprocess
from agent.tasks import ActionSpace
from agent.tasks import DatasetCompleteException
from agent.tasks import Task
from rclpy.node import Node
from std_srvs.srv import Empty
from std_msgs.msg import String
from r2pbi_commif.srv import CommandWithPath, CommandLookUpDist
import rclpy
import threading

def extract_tree(text):
    match = re.search(r'```(.*?)```', text, re.DOTALL)
    if match:
        extracted_text = match.group(1)
        return extracted_text
    else:
        print("No text found between triple backticks.")
        return extracted_text

class Example_Task(Task):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.observation_space = NotImplemented
        self.action_space = ActionSpace.CONTINUOUS
        self.args = kwargs
        self.episode_counter = 0

        # init ros node and start the environment
        rclpy.init()
        self.node = Node("sim_env_node")
        thread = threading.Thread(target=self.launch_env)
        thread.start()

        # define the client to access env
        self.reset_client, self.start_video_client, self.end_video_client, self.dist_of_obj_client = self.start_clients()

        # define gripper object to judge if final state is good 
        self.obj_on_gripper_ = ''
        object_ongripper_subscriber = self.node.create_subscription(
            String,
            '/robot/object_on_gripper',
            self.listener_callback,
            10)
        object_ongripper_subscriber


    def reset(self, next_subtask: str | None = None):
        self.done = False
        
        self.text_obs = 'pick up an red box.'

    def get_observation(self):
        return {"text": self.text_obs, "_available_actions": ""}


    def is_complete(self):
        return self.done
    
    def parse_method(self, raw_response: str):
        try:
            proposed_answer = extract_tree(raw_response)
        except Exception as e:
            proposed_answer = f"{e}"
        return proposed_answer

    def step(self, action):
        package_name = 'btcpp_machine'
        node_name = 'run_bt_client'
        parameter_name = 'bt_xml'
        self.done = True
        try:
            command = [
            'ros2', 'run', package_name, node_name,
            '--ros-args', '-p', f'{parameter_name}:={action}'
            ]
            self.node.get_logger().info("reset sim")
            future = self.reset_client.call_async(Empty.Request())
            rclpy.spin_until_future_complete(self.node, future, timeout_sec=20.0)

            self.node.get_logger().info("start test")
            # Run the command and capture output
            process = subprocess.run(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True, timeout=60)
            self.text_obs = process.stderr

            # calculate reward 
            self.node.get_logger().info("calculate reward.")
            reward = self.calculate_reward()
        except Exception:
            reward = self.calculate_reward()
        return reward, self.done
    
    def launch_env(self):
        package_name = 'r2pbi_env'
        launch_name = 'v2prototype.launch.py'
        command = [
            'ros2', 'launch', package_name, launch_name
        ]
        while True:
            print("start launch...")
            process = subprocess.run(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
            print("launch process dead, restart.....")

    def start_clients(self):
        reset_client = self.node.create_client(Empty, '/simulation/reset_sim')
        while not reset_client.wait_for_service(timeout_sec=1.0):
                self.node.get_logger().info('Service not available, waiting again...')
        start_video_client = self.node.create_client(CommandWithPath, '/simulation/record_video_start')
        while not start_video_client.wait_for_service(timeout_sec=1.0):
                self.node.get_logger().info('Service start_video not available, waiting again...')
        end_video_client = self.node.create_client(CommandWithPath, '/simulation/record_video_end')
        while not end_video_client.wait_for_service(timeout_sec=1.0):
                self.node.get_logger().info('Service end_video not available, waiting again...')
        dist_of_obj_client = self.node.create_client(CommandLookUpDist, '/robot/distance_to_object')
        while not dist_of_obj_client.wait_for_service(timeout_sec=1.0):
                self.node.get_logger().info('Service not available, waiting again...')
        return reset_client, start_video_client, end_video_client, dist_of_obj_client
    
    def listener_callback(self, msg):
        self.obj_on_gripper_ = msg.data
    
    def calculate_reward(self):
        # check if the action is correct
        reward = 0
        self.node.get_logger().info(f"after tree execution, object on gripper is {self.obj_on_gripper_}")
        if self.obj_on_gripper_ == "red_box":
            reward += 10
        
        # calculate distance to target 
        future = self.dist_of_obj_client.call_async(CommandLookUpDist.Request(object = 'red_box'))
        rclpy.spin_until_future_complete(self.node, future, timeout_sec=20.0)
        reward -= future.result().distance.data *100.0
        return reward

### Step 3: Define prompt template

We now create a new directory to save custom prompt templates for our example task. Within /src/agent/prompts/example_task we create .jinja files to represent prompt templates. Below we define three prompts: 
*    system_prompt.jinja: a prompt to define the task to the LLM
 *   context_example.jinja: a prompt that serves to give the agent contextual information as to how the environment works, used to direct the agent to generate related content.
  *  cot_prompt.jinja: used to get the agent to genrate a chain of thought and think step-by-step

In this example, we want to generate a behaviour tree using an LLM., and structure prompts to represent this.


In [ ]:
system_prompt = '''
[[ SYSTEM ]]
You are an agent that has to design a behaviour tree in xml format which follows BTCPP_format="4" standard with available actions and conditions.
You have to follow the instructions.
* Make sure the behaviour tree is surround by ``` ```
* Every round I will give you an observation and a list of available actions, objects and available places, you have to respond an action based on the state and task.
* You only have access to the Nodes/Actions/Conditions provided, you can not create new ones, this is a strict requirement.
* The behaviour tree must be in a xml format, follow BTCPP_format="4" standard.
* It may be the case that the task fails, you should include in the behaviour tree a case for recovering from potential issues.
* You must open gripper before grasp, and do not open gripper once grasp.
* You must navigate to find the object position.

{%- if cot_type in ["zero_shot_cot", "few_shot_cot"] %}
When asked for an action, you should think step by step, and then finish by providing an available action from the list.
Your response should be in the following format:
Thought: ...
Action: <action>
{%- else %}
When asked for an action, your response should be an available action from the list, using the following format:
Action: <action>
{%- endif %}
'''

context_example = '''
The available Nodes/Actions/Conditions/Objects/Places are presented below:

Tree Nodes:
{
root: root node should in format <root BTCPP_format="4">
BehaviorTree: <BehaviorTree> must be next layer of root
Sequence: A Sequence ticks all its children as long as they return SUCCESS. If any child returns FAILURE, the sequence is aborted.
Fallback: If a child returns FAILURE, the fallback ticks the next child. If the last child returns FAILURE too, all the children are halted and the fallback returns FAILURE. If a child returns SUCCESS, it stops and returns SUCCESS. All the children are halted. 
Inverter: tick the child once and return SUCCESS if the child failed or FAILURE if the child succeeded.
RetryUntilSuccessful: try number of times until subnode success
<RetryUntilSuccessful num_attempts="<max_number_of_attempt>">
}
'''

cot_prompt = '''
Now please think step by step, and then finish by providing behaviour tree that will help you complete the task.
Note: You can have multiple thoughts, but Only generate one behaviour tree.
Answer in the format
Thought: ...
Tree: ```<behaviour tree>```
'''

### Step 5: Define LLM server info
We must define an LLM backend for the agent to use. Within /configs/llm we create a file example_task_llm.yaml. Within this file, we specify the LLM and server IP that we want the agent to call.

In [ ]:
example_task_llm = '''
_target_: agent.llms.llm.OpenAIBackend
_partial_: true
server_ip: http://10.227.91.35:8000/v1
model_id: deepseek-coder-6.7b-instruct
api_key: EMPTY
max_response_tokens: 3000
'''

### Step 6: We test our custom example

In [ ]:
!python src/agent/start.py task=example_task method=fs-cot llm@_agents.0.llm=example_task